In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import logging 
import logging.config
import shutil
import sys
from itertools import cycle
from inspect import getfullargspec
from importlib import import_module
from time import sleep
from pathlib import Path
from distutils.util import strtobool
import waveshare_epd

In [2]:
import ArgConfigParse
from epdlib import Screen
from epdlib.Screen import Update
from library.CacheFiles import CacheFiles
from library.Plugin import Plugin
from library.InterruptHandler import InterruptHandler
from library import get_help
from library import run_module
import constants


In [3]:
def do_exit(status=0, message=None, **kwargs):
    if message:
        if status > 0:
            logging.error(f'failure caused exit: {message}')
        border = '\n'+'#'*70 + '\n'
        message = border + message + border + '\n***Exiting***'
        print(message)
        
    try:
        sys.exit(status)
    except Exception as e:
        pass

In [4]:
def clean_up(cache=None, screen=None):
    logging.info('cleaning up cache and screen')
    try:
        cache.cleanup()
    except AttributeError:
        logging.debug('no cache passed, skipping')
    try:
#         screen.initEPD()
        screen.clearEPD()
    except AttributeError:
        logging.debug('no screen passed, skipping')
    return

In [5]:
def get_cmd_line_args():
    cmd_args = ArgConfigParse.CmdArgs()
    cmd_args.add_argument('-c', '--config', ignore_none=True, metavar='CONFIG_FILE.ini',
                         type=str, dest='user_config',
                         help='use the specified configuration file')
    
    cmd_args.add_argument('-l', '--log_level', ignore_none=True, metavar='LOG_LEVEL',
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
                         dest='main__log_level', help='change the log output level')
    
    cmd_args.add_argument('--plugin_info', metavar='[plugin|plugin.function]',
                         required=False, default=None,
                         ignore_none=True,
                         help='get information for plugins and user-facing functions provided by a plugin')
    
    cmd_args.add_argument('--list_plugins', required=False,
                         default=False, action='store_true', 
                         help='list all available plugins')
    
    cmd_args.add_argument('--run_plugin_func',
                         required=False, default=None, nargs='+',
                         metavar=('plugin.function', 'optional_arg1 arg2 argN'),
                         ignore_none=True,
                         help='run a user-facing function for a plugin')
    
    cmd_args.add_argument('-d', '--daemon', required=False, default=False,
                         dest='main__daemon', action='store_true', 
                         help='run in daemon mode (ignore user configuration if found)')
    
    cmd_args.add_argument('-R', '--max_refresh', required=False, default=3, 
                          dest='main__max_refresh',
                          help='maximum number of refreshes between complete screen refresh')    
    
    cmd_args.add_argument('-V', '--version', required=False, default=False, ignore_false=True,
                          action='store_true',
                          help='display version and exit')
    

    
   
    cmd_args.parse_args()    
 
    return cmd_args


In [6]:
def get_config_files(cmd_args):
    '''read config.ini style file(s)
    Args:
        cmd_args(`ArgConfigParse.CmdArgs()` object)
    
    Returns:
        `ArgConfigParse.ConfigFile`'''
    config_files_list = [constants.config_base, constants.config_system, constants.config_user]
    
    daemon = False
    
    config_exists = True
    
    if hasattr(cmd_args.options, "main__daemon"):
        logging.debug('-d specified on command line')
        if cmd_args.options.main__daemon:
            logging.debug('excluding user config files')
            config_files_list.pop()
            daemon = True
        else:
            daemon = False
    else: 
        daemon = False
    
    if not daemon:
        # create a user config directory
        if not constants.config_user.exists():
            config_exists = False
            logging.info('creating user config directory and inserting config file')
            try:
                constants.config_user.parent.mkdir(parents=True, exist_ok=True)
            except PermissionError as e:
                logging.warning(f'could not create {constants.config_user}: {e}')
        if not constants.config_user.exists():
            try:
                shutil.copy(constants.config_base, constants.config_user)
            except Exception as e:
                logging.critical(f'could not copy configuration file to {constants.config_user}: {e}')
    
    if not config_exists:
        msg = f'''This appears to be the first time PaperPi has been run.
A user configuration file created: {constants.config_user}
At minimum you edit this file and add a display_type
        
Edit the configu file with "$ nano {constants.config_user}"'''
        do_exit(0, msg)
    
    logging.debug(f'using configuration files to configure PaperPi: {config_files_list}')
    config_files = ArgConfigParse.ConfigFile(config_files_list, ignore_missing=True)
    config_files.parse_config()

                    
    
    return config_files

In [7]:
# def sanitize_vals(config):
#     '''attempt to convert all the strings into appropriate formats
#         integer/float like strings ('7', '100', '-1.3') -> int
#         boolean like strings (yes, no, Y, t, f, on, off) -> 0 or 1
#     Args:
#         config(`dict`): nested config.ini style dictionary
    
#     Returns:
#         `dict`'''
    
#     def convert(d, func, exceptions):
#         '''convert values in nested dictionary using a specified function
#             values that raise an exception are left unchanged
        
#         Args:
#             d(`dict`): nested dictionary {'Section': {'key': 'value'}}
#             func(`function`): function such as int() or strtobool()
#             exceptions(`tuple`): 
            
#         Returns:
#             `dict`'''
#         for section, values in d.items():
#             for key, value in values.items():
#                 if isinstance(value, str):
#                     try:
#                         sanitized = func(value)
#                     except exceptions:
#                         sanitized = value
#                     d[section][key] = sanitized
#         return d
    
#     #attempt to convert any string that looks like int into the proper value
#     # convert strings to float if possible
#     str_to_float = convert(config, float, (ValueError))
#     str_to_ints = convert(str_to_float, int, (ValueError))
#     # attempt to convert any string that looks like a bool into a bool
#     str_to_bool = convert(str_to_ints, strtobool, (ValueError, AttributeError))
    
#     config = str_to_bool
    
    
        
#     return config

In [8]:
def sanitize_vals(config):
    '''attempt to convert all the strings into appropriate formats
             integer/float like strings ('7', '100', '-1.3') -> int or float
             boolean like strings (yes, no, Y, t, f, on, off) -> 0 or 1
         Args:
             config(`dict`): nested config.ini style dictionary

         Returns:
             `dict`'''    
    def strtofloat(s):
        '''strings to float if possible'''
        retval = s
        if isinstance(s, str):
            if '.' in s:
                try:
                    retval = float(s)
                except ValueError:
                    pass

        return retval

    def convert(d, new_type, exceptions):
        for section, values in d.items():
            for key, value in values.items():
                if isinstance(value, str):
                    try:
                        sanitized = new_type(value)
                    except exceptions:
                        sanitized = value

                    d[section][key] = sanitized
                else:
                    d[section][key] = value
        return d

    convert(config, strtofloat, ValueError)
    convert(config, int, (ValueError))
    convert(config, strtobool, (ValueError, AttributeError))
    
    return config

In [9]:
def setup_splash(config, resolution):
    logging.debug('checking splash settings')
    if 'splash' in config['main']:
        logging.debug('checking splash screen settings')
        if config['main']['splash']:
            logging.debug('splash enabled in confg file')
            splash = True
        else:
            logging.debug('splash disabled in config file')
            splash = False
    else:
        splash = True

    if splash:
        logging.debug('splash screen enabled')
        from plugins.splash_screen import splash_screen
        splash_config = { 
            'name': 'Splash Screen',
            'layout': splash_screen.layout.layout,
            'update_function': splash_screen.update_function,
            'resolution': resolution
        } 
        logging.debug(f'configuring splash screen: {splash_config}')
        splash = Plugin(**splash_config)
        splash.update(constants.app_name, constants.version, constants.url)
        logging.debug(f'splash screen image type: {type(splash.image)}')
    return splash

In [10]:
def setup_display(config):
    def ret_obj(obj=None, status=0, message=None):
        return{'obj': obj, 'status': status, 'message': message}    
    keyError_fmt = 'configuration KeyError: section[{}], key: {}'

    moduleNotFoundError_fmt = 'could not load module: {} -- error: {}'
    
#     try:
#         logging.debug('setting display type')
#         epd_module = '.'.join([constants.waveshare_epd, config['main']['display_type']])
#         epd = import_module(epd_module)
#     except KeyError as e:
#         return_val = ret_obj(obj=None, status=1, message=keyError_fmt.format('main', 'display_type'))
#         logging.error(return_val['message'])
#         return return_val
#     except ModuleNotFoundError as e:
#         logging.error('Check your config files and ensure a known waveshare_epd display is specified!')
#         return_val = ret_obj(None, 1, moduleNotFoundError_fmt.format(config["main"]["display_type"], e))
#         return return_val
#     except FileNotFoundError as e:
#         msg = f''''Error loading waveshare_epd module: {e}
#         This is typically due to SPI not being enabled, or the current user is 
#         not a member of the SPI group.
#         "$ sudo raspi-config nonint get_spi" will return 0 if SPI is enabled
#         Try enabling SPI and run this program again. '''
#         logging.error(msg)
#         return_val = ret_obj(obj=None, status=1, message=msg)
#         return return_val
    
    epd = config['main']['display_type']
    vcom = config['main']['vcom']
    screen = Screen(epd=epd, vcom=vcom)
    try:
        screen.epd = epd
    except PermissionError as e:
        logging.critical(f'Error initializing EPD: {e}')
        logging.critical(f'The user executing {constants.app_name} does not have access to the SPI device.')
        return_val = ret_obj(None, 1, 'This user does not have access to the SPI group\nThis can typically be resolved by running:\n$ sudo groupadd <username> spi')
        return return_val

    try:
        config['main']['rotation'] = int(config['main']['rotation'])
    except KeyError as e:
        logging.info(keyError_fmt.format('main', 'rotation'))
        logging.info('using default: 0')
    try:
        screen.rotation = config['main']['rotation']
    except ValueError as e:
        logging.error('invalid rotation; valid values are: 0, 90, -90, 180')
        return_val = ret_obj(None, 1, keyError_fmt.format('main', 'rotation'))
        
    return ret_obj(obj=screen)

In [11]:
def build_plugin_list(config, resolution, cache):
    # get the expected key-word args from the Plugin() spec
    spec_kwargs = getfullargspec(Plugin).args

    plugins = []

    for section, values in config.items():
        # ignore the other sections
        if section.startswith('Plugin:'):
            logging.info(f'[[ {section} ]]')

            my_config = {}
            # add all the spec_kwargs from the config
            plugin_kwargs = {}
            for key, val in values.items():
                if key in spec_kwargs:
                    my_config[key] = val
                else:
                    # add everything that is not one of the spec_kwargs to this dict
                    plugin_kwargs[key] = val

            # populate the kwargs my_config dict that will be passed to the Plugin() object
            my_config['name'] = section
            my_config['resolution'] = resolution
            my_config['config'] = plugin_kwargs
            my_config['cache'] = cache
            try:
                module = import_module(f'{constants.plugins}.{values["plugin"]}')
                my_config['update_function'] = module.update_function
                my_config['layout'] = getattr(module.layout, values['layout'])
            except KeyError as e:
                logging.info('no module specified; skipping update_function and layout')
            except ModuleNotFoundError as e:
                logging.warning(f'error: {e} while loading module {constants.plugins}.{values["plugin"]}')
                logging.warning(f'skipping module')
                continue
            my_plugin = Plugin(**my_config)
            try:
                my_plugin.update()
            except AttributeError as e:
                logging.warning(f'ignoring plugin {my_plugin.name} due to missing update_function')
                logging.warning(f'plugin threw error: {e}')
                continue
            logging.info(f'appending plugin {my_plugin.name}')
            plugins.append(my_plugin)
            
            
    if len(plugins) < 1:
        my_config = {}
        logging.warning('no plugins were loaded! falling back to default plugin.')
        my_config['name'] = 'default plugin'
        my_config['resolution'] = resolution
        my_config['cache'] = cache
        try:
            module = import_module(f'{constants.plugins}.default')
        except ModuleNotFoundError as e:
            msg = f'could not load {constants.plugins}.default'
            logging.error(msg)
            do_exit(1, msg)
        my_config['update_function'] = module.update_function
        my_config['layout'] = getattr(module.layout, 'default')
        my_plugin = Plugin(**my_config)
        plugins.append(my_plugin)
        
    return plugins

In [12]:
def update_loop(plugins, screen, max_refresh=2):
    exit_code = 1
    logging.info('starting update loop')
    
    
    def update_plugins(): 
        '''run through all active plugins and update while recording the priority'''
        my_list = []
        logging.info('*'*15)
        logging.info(f'My PID: {os.getppid()}')
        logging.info(f'updating {len(plugins)} plugins')
        for plugin in plugins:
            plugin.update()
            logging.debug(f'update: [{plugin.name}]-p: {plugin.priority}')
            my_list.append(plugin.priority)
        logging.debug(f'priorities: {my_list}')
        return my_list
        
    # use itertools cycle to move between list elements
    plugin_cycle = cycle(plugins)
    plugin_is_active = False
    # current plugin for display
    this_plugin = next(plugin_cycle)
    # track time plugin is displayed for
    this_plugin_timer = Update()
    # each plugin generates a unique hash whenever it is updated
    this_hash = ''
    
    # update all the plugins and record priority
    priority_list = update_plugins()
    # this var name is confusing -- it's actually the lowest number to indicate **maximum** priority
    max_priority = min(priority_list)
    # record for comparison
    last_priority = max_priority
    
    # count the number of refreshes for HD Screens
    refresh_count = 0    
    
    logging.info(f'max_priority: {max_priority}')
    
    
    
    for plugin in plugins:
        if plugin.priority <= max_priority:
            this_hash = plugin.hash
            logging.info(f'**** displaying {plugin.name} ****')
#             screen.initEPD()
            screen.writeEPD(plugin.image)
            break
    
    

    
    with InterruptHandler() as h:
        while True:    
            if h.interrupted:
                logging.info('caught interrupt -- stoping execution')
                exit_code = 0
                break

            priority_list = update_plugins()

            # priority increases as it gets lower; 0 is considered the bottom,
            # but some modules may temporarily have a negative priority to indicate a critical
            # update 
            last_priority = max_priority
            max_priority = min(priority_list)
            
            logging.debug(f'{this_plugin.name}: last updated: {this_plugin_timer.last_updated}, min_display_time: {this_plugin.min_display_time}')
            
            
            # if the timer has expired OR a module has changed the priority setting begin the update procedure
            if this_plugin_timer.last_updated > this_plugin.min_display_time or max_priority < last_priority:
                logging.info(f'plugin expired -- switching plugin')
                plugin_is_active = False
                
                # cycle through plugins, looking for the next plugin that has high priority
                while not plugin_is_active:
                    this_plugin = next(plugin_cycle)
                    logging.debug(f'checking priority of {plugin.name}')
                    if this_plugin.priority <= max_priority:
                        plugin_is_active = True
                    else:
                        logging.debug('trying next plugin')
                        pluggin_is_active = False
                        
                logging.info(f'displaying {this_plugin.name} -- priority: {this_plugin.priority}/{max_priority}')
                
                if this_hash != this_plugin.hash:
                    logging.debug('data refreshed, refreshing screen')
                    this_hash = this_plugin.hash
#                     screen.initEPD()
                    logging.debug(f'image type: {type(this_plugin.image)}')
    
                    # wipe screen if the max_refresh count is exceeded
                    if refresh_count > max_refresh:
                        refresh_count = 0
                        if screen.HD:
                            logging.info('max_refresh exceeded, wiping screen prior to next update')
                            screen.clearEPD()
    
                    if screen.writeEPD(this_plugin.image):
                        logging.debug('successfully wrote image')
                        refresh_count += 1
                    else:
                        logging.warning('#=#=# failed to write image #=#=#')
                        logging.info('trying next plugin')
                        plugin_is_active = False
                        
                    
                else:
                    logging.debug('plugin data not refreshed -- skipping screen refresh')
                this_plugin_timer.update()    
                    
        
            sleep(1)
    return exit_code

In [13]:
def main():
    
    # change the working directory -- this simplifies all path work later on
    os.chdir(constants.absolute_path)
    
    # set the absolute path to the current directory
    absolute_path = constants.absolute_path
       
    # set up logging
    logging.config.fileConfig(constants.logging_config)
    logger = logging.getLogger(__name__)
    
    # get command line and config file arguments
    cmd_args = get_cmd_line_args()
    
    if hasattr(cmd_args, 'unknown'):
        print(f'Unknown arguments: {cmd_args.unknown}\n\n')
        cmd_args.parser.print_help()
        return
        
    
    config_files = get_config_files(cmd_args)
    
    # merge file and commandline (right-most over-writes left)
    config = ArgConfigParse.merge_dict(config_files.config_dict, cmd_args.nested_opts_dict)
    
    if cmd_args.options.version:
        print(constants.version_string)
        return
    
    if cmd_args.options.plugin_info:
        print(get_help.get_help(cmd_args.options.plugin_info))
        return
    
    if cmd_args.options.list_plugins:
        print(get_help.get_help())
        return
    
    if cmd_args.options.run_plugin_func:
        run_module.run_module(cmd_args.options.run_plugin_func)
        return
    
    # make sure all the integer-like strings are converted into integers
    config = sanitize_vals(config)
#     return config
    
    
    logger.setLevel(config['main']['log_level'])
    logging.root.setLevel(config['main']['log_level'])
    
    logging.debug(f'********** PaperPi {constants.version} Starting **********')
    
    # configure screen
    screen_return = setup_display(config)
    if screen_return['obj']:
        screen = screen_return['obj']
    else:
        clean_up(None, None)
        logging.error(f'config files used: {config_files.config_files}')
        do_exit(**screen_return)
    
    # try to set up the splash screen several times here -- this may solve the None image problem.
    splash = setup_splash(config, screen.resolution)
    
    if splash:
        
        logging.debug('displaying splash screen')
        logging.debug(f'image type: {type(splash.image)}')
        screen.writeEPD(splash.image)
        
    
    cache = CacheFiles(path_prefix=constants.app_name)
    plugins = build_plugin_list(config, screen.resolution, cache)
    
    exit_code = update_loop(plugins, screen)

    logging.info('caught terminate signal -- cleaning up and exiting')
    clean_up(cache, screen)
    
    return exit_code

In [14]:
if __name__ == "__main__":
    # remove jupyter runtime junk for testing
    if len(sys.argv) >= 2 and 'ipykernel' in sys.argv[0]:
        sys.argv = [sys.argv[0]]
        sys.argv.extend(sys.argv[3:])
    exit_code = main()
    sys.exit(exit_code)

22:26:11 1239797774:main:51:DEBUG - ********** PaperPi 0.2.00 Starting **********
22:26:13 Screen:epd:276:DEBUG - epd configuration: {'epd': <IT8951.display.AutoEPDDisplay object at 0xae915fd0>, 'resolution': [1200, 825], 'clear_args': {}, 'one_bit_display': False, 'constants': <module 'IT8951.constants' from '/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/IT8951/constants.py'>}
22:26:13 Screen:rotation:241:DEBUG - rotation=0, resolution=[1200, 825]
22:26:16 Screen:epd:276:DEBUG - epd configuration: {'epd': <IT8951.display.AutoEPDDisplay object at 0xae915cd0>, 'resolution': [1200, 825], 'clear_args': {}, 'one_bit_display': False, 'constants': <module 'IT8951.constants' from '/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/IT8951/constants.py'>}
22:26:16 Screen:rotation:241:DEBUG - rotation=0, resolution=[1200, 825]
22:26:16 3124529230:setup_splash:2:DEBUG - checking splash settings
22:26:16 3124529230:setup_splash:4

22:26:17 Block:__init__:150:DEBUG - creating Block
22:26:17 Block:font:415:DEBUG - setting old_font = None
22:26:17 Block:_calc_maxchar:504:DEBUG - calculating maximum characters for font ('Dosis', 'SemiBold')
22:26:17 Block:_calc_maxchar:518:DEBUG - maximum characters per line: 28
22:26:17 Block:_text_formatter:534:DEBUG - formatting string: .
22:26:17 Block:_text_formatter:537:DEBUG - formatted list:
 ['.']
22:26:17 Block:_text2image:587:DEBUG - scaling image to fit in padded area
22:26:17 Block:_text2image:592:DEBUG - paste_y: 81
22:26:17 Layout:update_contents:412:INFO - updating blocks
22:26:17 Layout:update_contents:418:DEBUG - updating block: app_name
22:26:17 Block:_text_formatter:534:DEBUG - formatting string: PaperPi
22:26:17 Block:_text_formatter:537:DEBUG - formatted list:
 ['PaperPi']
22:26:17 Block:_text2image:587:DEBUG - scaling image to fit in padded area
22:26:17 Block:_text2image:592:DEBUG - paste_y: 46
22:26:17 Layout:update_contents:418:DEBUG - updating block: versi

22:26:19 Layout:_check_keys:401:DEBUG - layout keys: {'image': True, 'max_lines': None, 'padding': 5, 'width': 0.3333333333333333, 'height': 0.5714285714285714, 'abs_coordinates': (0, 0), 'hcenter': True, 'vcenter': True, 'relative': False, 'mode': 'L', 'rand': False, 'inverse': False, 'font': None, 'font_size': None, 'maxchar': None, 'dimensions': None, 'scale_x': None, 'scale_y': None, 'fill': 0, 'bkground': 255}
22:26:19 Layout:_calculate_layout:242:DEBUG - dimensions: (400, 471)
22:26:19 Layout:_calculate_layout:269:DEBUG - section has absolute coordinates
22:26:19 Layout:_calculate_layout:271:DEBUG - coordinates: (0, 0)
22:26:19 Layout:_calculate_layout:236:INFO - *****title*****
22:26:19 Layout:_check_keys:393:DEBUG - checking layout keys
22:26:19 Layout:_check_keys:401:DEBUG - layout keys: {'image': None, 'max_lines': 2, 'padding': 4, 'width': 0.6666666666666666, 'height': 0.5714285714285714, 'abs_coordinates': (None, 0), 'relative': ['coverart', 'title'], 'hcenter': True, 'vcen

22:26:20 Layout:update_contents:418:DEBUG - updating block: artist
22:26:20 Block:_text_formatter:534:DEBUG - formatting string: Err: no data
22:26:20 Block:_text_formatter:537:DEBUG - formatted list:
 ['Err: no data']
22:26:20 Block:_text2image:587:DEBUG - scaling image to fit in padded area
22:26:20 Block:_text2image:592:DEBUG - paste_y: 75
22:26:20 Layout:update_contents:418:DEBUG - updating block: album
22:26:20 Block:_text_formatter:534:DEBUG - formatting string: Err: no data
22:26:20 Block:_text_formatter:537:DEBUG - formatted list:
 ['Err: no data']
22:26:20 Block:_text2image:587:DEBUG - scaling image to fit in padded area
22:26:20 Block:_text2image:592:DEBUG - paste_y: 17
22:26:20 Layout:update_contents:421:DEBUG - ignoring block artwork_url
22:26:20 Layout:update_contents:421:DEBUG - ignoring block duration
22:26:20 Layout:update_contents:421:DEBUG - ignoring block player
22:26:20 Layout:update_contents:421:DEBUG - ignoring block mode
22:26:20 Layout:concat:436:DEBUG - concati

22:26:20 Layout:_calculate_layout:251:DEBUG - section has calculated position
22:26:20 Layout:_calculate_layout:271:DEBUG - coordinates: (0, 0)
22:26:20 Layout:_scalefont:299:DEBUG - calculating maximum font size for area: (300, 248)
22:26:20 Layout:_scalefont:300:DEBUG - using font: /home/pi/src/epd_display/paperpi/fonts/Economica/Economica-Bold.ttf
22:26:20 Layout:_scalefont:309:DEBUG - target X font dimension 352.94117647058823
22:26:20 Layout:_scalefont:310:DEBUG - target Y font dimension 186.0
22:26:20 Layout:_scalefont:325:DEBUG - X target reached
22:26:20 Layout:_scalefont:333:DEBUG - test string: W W W ; pixel dimensions for fontsize 141: (354, 135)
22:26:20 Layout:_calculate_layout:236:INFO - *****006_forecast_time_local*****
22:26:20 Layout:_check_keys:393:DEBUG - checking layout keys
22:26:20 Layout:_check_keys:401:DEBUG - layout keys: {'image': None, 'inverse': True, 'padding': 0, 'width': 0.25, 'height': 0.3, 'hcenter': False, 'vcenter': True, 'abs_coordinates': (0, None),

...

22:26:57 word_clock:update_function:94:DEBUG - using 22:26
22:26:57 Layout:update_contents:412:INFO - updating blocks
22:26:57 Layout:update_contents:418:DEBUG - updating block: wordtime
22:26:57 Block:_text_formatter:534:DEBUG - formatting string: It's round about Thirty Past Ten
22:26:57 Block:_text_formatter:537:DEBUG - formatted list:
 ["It's round about", 'Thirty Past Ten']
22:26:57 Block:_text2image:587:DEBUG - scaling image to fit in padded area
22:26:57 Layout:update_contents:418:DEBUG - updating block: time
22:26:57 Block:_text_formatter:534:DEBUG - formatting string: 22:26
22:26:57 Block:_text_formatter:537:DEBUG - formatted list:
 ['22:26']
22:26:57 Block:_text2image:587:DEBUG - scaling image to fit in padded area
22:26:57 Layout:concat:436:DEBUG - concating blocks into single image
22:26:57 Layout:concat:438:DEBUG - pasitng **wordtime** image at: (0, 0)
22:26:57 Layout:concat:438:DEBUG - pasitng **time** image at: (0, 707)
22:26:57 3309868768:update_plugins:14:INFO - update

22:27:04 3309868768:update_plugins:14:INFO - update: [Plugin: Word Clock]-p: 2
22:27:04 Plugin:_is_ready:209:DEBUG - throttling in effect -- wait for 7.87542015100189 seconds before requesting update
22:27:04 3309868768:update_plugins:14:INFO - update: [Plugin: LibreSpot]-p: 32768
22:27:04 Plugin:_is_ready:209:DEBUG - throttling in effect -- wait for 17.713792971990188 seconds before requesting update
22:27:04 3309868768:update_plugins:14:INFO - update: [Plugin: Weather Den Haag]-p: 2
22:27:04 3309868768:update_plugins:16:INFO - priorities: [2, 32768, 2]
22:27:04 3309868768:update_loop:69:DEBUG - Plugin: Word Clock: last updated: 35.70413151101093, min_display_time: 60
22:27:05 3309868768:update_plugins:9:INFO - ***************
22:27:05 3309868768:update_plugins:10:INFO - My PID: 5849
22:27:05 3309868768:update_plugins:11:INFO - updating 3 plugins
22:27:05 Plugin:_is_ready:209:DEBUG - throttling in effect -- wait for 0.5746298320009373 seconds before requesting update
22:27:05 33098687

22:27:11 3309868768:update_plugins:14:INFO - update: [Plugin: Weather Den Haag]-p: 2
22:27:11 3309868768:update_plugins:16:INFO - priorities: [2, 32768, 2]
22:27:11 3309868768:update_loop:69:DEBUG - Plugin: Word Clock: last updated: 43.06171109700517, min_display_time: 60
22:27:12 3309868768:update_plugins:9:INFO - ***************
22:27:12 3309868768:update_plugins:10:INFO - My PID: 5849
22:27:12 3309868768:update_plugins:11:INFO - updating 3 plugins
22:27:12 Plugin:_is_ready:209:DEBUG - throttling in effect -- wait for 2.67545747999975 seconds before requesting update
22:27:12 3309868768:update_plugins:14:INFO - update: [Plugin: Word Clock]-p: 2
22:27:12 librespot_client:update_function:81:DEBUG - update_function for plugin Plugin: LibreSpot, version 0.1.1
22:27:12 librespot_client:update_function:98:DEBUG - fetching API access token from librespot player SpoCon-Spotify
22:27:12 librespot_client:update_function:99:DEBUG - requesting spotify API access scope: user-read-playback-state
2

SystemExit: 0

/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
!jupyter-nbconvert --to python --template python_clean paperpi.ipynb

[NbConvertApp] Converting notebook paperpi.ipynb to python
[NbConvertApp] Writing 22269 bytes to paperpi.py


In [ ]:
# logger = logging.getLogger(__name__)
# logger.root.setLevel('DEBUG')